<a href="https://colab.research.google.com/github/ehdghks316/itwill_studyGroup_D/blob/main/3_30%ED%8C%8C%EC%9D%B4%EC%8D%AC(sqlite2).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import sqlite3
conn = sqlite3.connect('c:/data/insa.db')
c = conn.cursor()
c.execute('PRAGMA table_info("emp")')
c.execute('PRAGMA table_info("dept")')
c.fetchall()

[문제165] 부서별 최고 급여자들에 대한 정보를 출력해주세요.
c.execute('''select *
          from (select department_id, max(salary) m_sal
                from emp
                group by department_id) e1 join emp e2
          where e2.department_id = e1.department_id
          and e2.salary = e1.m_sal''')
c.fetchall()

c.execute('''select *
          from (select *, max(salary) m_sal
                from emp
                group by department_id)
          where salary = m_sal''')
c.fetchall()

c.execute('''select employee_id, last_name, salary, department_id
          from emp
          where (department_id, salary) in (select department_id, max(salary)
                                            from emp
                                            group by department_id)
          order by 4''')
c.fetchall()

c.execute('''with 
          dept_max as (select department_id, max(salary) max_sal
                       from emp
                       group by department_id)
          select department_id, last_name, salary
          from emp
          where (department_id, salary) in (select department_id, max_sal from dept_max)''')
c.fetchall() # with문 쓸 때 주의해서 사용 (지금의 경우는 성능상 좋지않음)

c.execute('''select *
          from (select employee_id, department_id, salary,
                case when salary = max(salary) over(partition by department_id) then "o" end label
                from emp) 
          where label = "o" ''')
c.fetchall()

c.execute('''select employee_id, department_id, last_name, salary
          from (select employee_id, department_id, last_name, salary,
          case when dense_rank() over(partition by department_id order by salary desc) = 1 then 1 end label
          from emp)
          where label = 1''')          
c.fetchall()

[문제166] 부서별 최소 급여자들에 대한 정보를 출력해주세요.
c.execute('''select *
          from (select department_id, min(salary) m_sal
                from emp
                group by department_id) e1 join emp e2
          where e2.department_id = e1.department_id
          and e2.salary = e1.m_sal''')
c.fetchall()

c.execute('''select *
          from (select employee_id, department_id, salary, min(salary) over(partition by department_id),
                case when salary = min(salary) over(partition by department_id) then "o" end label
                from emp) 
          where label = "o" ''')
c.fetchall()


c.execute('''select employee_id, last_name,salary, rank() over(order by salary desc) from emp''')
c.fetchall()

c.execute('''select employee_id, last_name,salary, dense_rank() over(order by salary desc) from emp''')
c.fetchall()


c.execute('''select employee_id, last_name,salary, rank() over(partition by department_id order by salary desc) from emp''')
c.fetchall()

c.execute('''select employee_id, last_name,salary, dense_rank() over(partition by department_id  order by salary desc) from emp''')
c.fetchall()

c.execute('''select employee_id, department_id, last_name, salary
          from (select employee_id, department_id, last_name, salary,
          case when dense_rank() over(partition by department_id order by salary) = 1 then 1 end label
          from emp)
          where label = 1''')          
c.fetchall()

[문제167] 근무일수가 가장 많은 10위까지 직원들의 employee_id, last_name, department_name, 근무일수를 출력해주세요.
단 sqlite 이용하세요.
c.execute('''select employee_id, last_name, department_name, work_day
          from (select employee_id, last_name, department_id, ((strftime("%s",date("now")) - strftime("%s",date(hire_date)))/86400) work_day,
                dense_rank() over(order by (strftime("%s",date("now")) - strftime("%s",date(hire_date)))/86400 desc) w_day
                from emp) e join dept d
          using(department_id)
          where w_day <= 10''')
c.fetchall()


# 강사님
c.execute('''select e.employee_id, e.last_name, d.department_name, e.working_day, e.rank
          from (select *
                from (select employee_id, last_name, department_id, dense_rank() over(order by working_day desc) as rank, working_day
                      from (select employee_id, last_name, department_id,
                            (strftime("%s","now") - strftime("%s",date(hire_date)))/86400 working_day
                            from emp))
                where rank <= 10) e, dept d
            where e.department_id = d.department_id
            order by 5''')
c.fetchall()

[문제168] 근무일수가 가장 많은 10위까지 직원들의 employee_id, last_name, department_name, 근무일수를 출력해주세요.
단 pandas 이용하세요.
from pandas import DataFrame, Series
import pandas as pd

emp = pd.read_csv('c:/data/employees.csv')
dept = pd.read_csv('c:/data/departments.csv')

emp['근무일수'] = (pd.Timestamp.now() - pd.to_datetime(emp['HIRE_DATE'])).dt.days
wday_emp = emp[emp['근무일수'].rank(ascending=False, method='dense') <= 10]

pd.merge(wday_emp,dept, on='DEPARTMENT_ID')[['EMPLOYEE_ID','LAST_NAME','DEPARTMENT_NAME','근무일수']]

# 강사님
x = DataFrame({'사번':emp.EMPLOYEE_ID,
           '이름':emp.LAST_NAME,
           '부서코드':emp.DEPARTMENT_ID,
           '근무일수':(pd.Timestamp.now() - pd.to_datetime(emp.HIRE_DATE,format='%Y-%m-%d')).dt.days})
x['순위'] = x['근무일수'].rank(ascending=False,method='dense').astype(int)
top_10 = x.loc[x['순위'] <= 10].sort_values(by='순위')
pd.merge(top_10,dept,left_on = '부서코드',right_on='DEPARTMENT_ID')[['사번','이름','근무일수','순위','DEPARTMENT_NAME']].sort_values(by='순위')


v_id = 1
v_name = 'james'
v_sal = 1000

c.execute('create table insa(id integer,name char, sal integer)')
c.execute('insert into insa(id,name,sal) values(?,?,?)',(1,'james',1000))
c.execute('select * from insa')
c.fetchone()
conn.rollback()

# 변수에 있는 값을 테이블에 입력하는 방법 (물음표? 사용)
#c.execute('insert into insa(id,name,sal) values(1,'james',1000)')
c.execute('insert into insa(id,name,sal) values(?,?,?)',(v_id,v_name,v_sal))
c.execute('select * from insa')
c.fetchall()
conn.commit()

c.execute('select * from insa where id = ?',(1,))
c.fetchall()

c.execute('select * from insa where id = ?',(v_id,))
c.fetchall()

c.execute('update insa set name = ? where id = ?',('제임스',1))
c.execute('select * from insa')
c.fetchall()
conn.rollback()
c.execute('select * from insa')
c.fetchall()

v_id = 1
v_name = '홍길동'
c.execute('update insa set name = ? where id = ?',(v_name,v_id))
c.execute('select * from insa')
c.fetchall()

c.execute('delete from insa where id = ?', (v_id,))
c.execute('select * from insa')
c.fetchall()
conn.rollback()
c.execute('select * from insa')
c.fetchall()


[문제 169] 주소록 관리 프로그램 만들기. Contact 클래스에 
open, close, search, insert, update, delete, commit, rollback 함수를 구현해주세요.
'''
c = Contact()
c.open()
c.insert('홍길동','010-1234-5678','abc@naver.com','경기도 시흥시')
c.search('010-1234-5678')
c.commit()
c.update('010-1234-5678', "james@naver.com") # 메일정보 수정
c.close()
'''
import sqlite3
from pandas import Series, DataFrame
import pandas as pd

# 테이블 만들기
conn = sqlite3.connect('c:/data/insa.db')
c = conn.cursor()
c.execute('create table contact(name char, p_num char, email char, addr char)')
c.execute('select * from contact')
c.execute('PRAGMA table_info(contact)')
c.fetchall()

c.execute('insert into contact(name,p_num,email,addr) values(?,?,?,?)',('그리유','010-1234-5678','aaa@aaa.com','경기도')) # 테스트
c.execute('select * from contact')
c.fetchall()
conn.rollback()

#open, close, search, insert, update, delete, commit, rollback 함수 구현
conn = None
cs = None

def open():
    global conn
    global cs
    import sqlite3
    conn = sqlite3.connect('c:/data/insa.db')
    cs = conn.cursor()
    print('insa db파일을 연결했습니다.')

open()

def insert(name,p_num,email,addr):
    global cs
    cs.execute('insert into contact(name,p_num,email,addr) values(?,?,?,?)',(name,p_num,email,addr))
    print("이름 : {}, 전화번호 : {}, 이메일 : {}, 주소 : {} \n데이터가 삽입되었습니다.".format(name,p_num,email,addr))
insert('홍길동','010-1000-0001','aaa@aaa.com','경기도')
insert('김두한','010-2000-0002','bbb@bbb.com','서울')    

def search(p_num):
    global cs
    cs.execute('select * from contact where p_num = ?',(p_num,))
    print(cs.fetchall())
search('010-2000-0002')
search('010-1000-0001')

def update(p_num,email):
    global cs
    cs.execute('update contact set email = ? where p_num = ?',(email,p_num))
    print('{}의 이메일이 {} 로 변경 완료'.format(p_num,email))
update('010-1000-0001','ccc@ccc.com')

def delete(p_num):
    global cs
    cs.execute('delete from contact where p_num = ?',(p_num,))
    print(p_num,'삭제 완료')
delete('010-2000-0002')

def commit():
    global conn
    conn.commit()
    print('커밋 완료')
commit()

def rollback():
    global conn
    conn.rollback()
    print('롤백 완료')
rollback()

def close():
    global conn
    global cs
    cs.close()
    print('cursor 종료')
    conn.close()
    print('connect 종료')
    
# class 만들기
class Contact:
    
    def open(self): # 디비 연결 함수
        import sqlite3
        self.conn = sqlite3.connect('c:/data/insa.db')
        self.cs = self.conn.cursor()
        print('insa db파일을 연결했습니다.')

    def insert(self,name,p_num,email,addr): # 데이터 삽입
        self.cs.execute('insert into contact(name,p_num,email,addr) values(?,?,?,?)',(name,p_num,email,addr))
        print("이름 : {}, 전화번호 : {}, 이메일 : {}, 주소 : {} \n데이터가 삽입되었습니다.".format(name,p_num,email,addr))

    def search(self,p_num): # 데이터 검색
        self.cs.execute('select * from contact where p_num = ?',(p_num,))
        print(self.cs.fetchall())
        
    def search_all(self): # 모든 데이터 검색
        self.cs.execute('select * from contact')
        print(self.cs.fetchall())
    
    def length(self):
        self.cs.execute('select count(*) from contact')
        print(self.cs.fetchall(),"개의 데이터가 존재합니다")
        
    def update(self,p_num,email): # 데이터 수정
        self.cs.execute('update contact set email = ? where p_num = ?',(email,p_num))
        print('{}의 이메일이 {} 로 변경 완료'.format(p_num,email))

    def delete(self,p_num): # 데이터 삭제
        self.cs.execute('delete from contact where p_num = ?',(p_num,))
        print(p_num,'삭제 완료')

    def commit(self): # 커밋(수행 결과 저장)
        self.conn.commit()
        print('커밋 완료')
        
    def rollback(self): # 롤백(되돌리기)
        self.conn.rollback()
        print('롤백 완료')

    def close(self): # 종료
        self.cs.close()
        print('cursor 종료')
        self.conn.close()
        print('connect 종료')
    

# 수행
insert('홍길동','010-1000-0001','aaa@aaa.com','경기도')
insert('김두한','010-2000-0002','bbb@bbb.com','서울')
insert('가자미','010-3000-0003','ddd@aaa.com','제주도')
insert('김사랑','010-4000-0004','eee@bbb.com','경상도')    
search('010-2000-0002')
search('010-1000-0001')
search('010-3000-0003')
search('010-4000-0004')
rollback()
commit()
update('010-4000-0004','fff@gmail.com')
delete('010-2000-0002')
close()

c = Contact()
c.open()
c.insert('김민혁','010-0000-0001','ggg@gmail.com','충청도')
c.search('010-1000-0001')
c.search('010-0000-0001')
c.search_all()
c.length()
c.update('010-1000-0001','zzz@gmail.com')
c.close()


### 강사님 방법
import sqlite3
conn = sqlite3.connect('c:/data/contact.db')
cur = conn.cursor()
cur.execute('create table contact(name text, pn text, email text, addr text)')
cur.close()
conn.close()

class Contact1:
    
    def open(self):
        self.conn = sqlite3.connect('c:/data/contact.db')
        self.cur = self.conn.cursor()
        print('contact db에 접속했습니다.')
        
    def close(self):
        self.cur.close()
        self.conn.close()
        print('contact db 접속 해지했습니다.')
        
    def search(self,pn):
        try:
            self.cur.execute('select * from contact where pn = ?',(pn,))
            self.lst = self.cur.fetchall()
            if len(self.lst) == 0:
                print("데이터가 없습니다.")
            else:
                print(self.lst)
        except:
            print("open 함수를 호출한 후 수행하세요.")
    
    def insert(self,name,pn,email,addr):
        insert_sql = 'insert into contact(name,pn,email,addr) values(?,?,?,?)'
        self.cur.execute(insert_sql,(name,pn,email,addr))
    
    def commit(self):
        self.conn.commit()
        
    def rollback(self):
        self.conn.rollback()
        
    def update(self,pn,email):
        self.cur.execute('update contact set email = ? where pn = ?',(email,pn))
    
    def delete(self,pn):
        self.cur.execute('delete from contact where pn = ?',(pn,))

c = Contact1()
c.open()
c.insert('홍길동','010-1000-0004','hong@aaa.com','서울시 강남구 삼성로')
c.search('010-1000-0004')
c.commit()

c.update("010-1000-0004","james@aaa.com")
c.delete("010-1000-0004")
c.rollback()
c.close()
